In [1]:
import sqlite3

In [4]:
import sqlite3
import random
import string
from datetime import datetime, timedelta

# Connect to the SQLite database
conn = sqlite3.connect('transactions.db')
cursor = conn.cursor()

# Step 1: Create the 'users' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL
    )
''')

# Generate a random name (composed of random letters)
def random_name(length=7):
    return ''.join(random.choices(string.ascii_letters, k=length))

# Insert 50 random users into the 'users' table
NUM_USERS = 50

for _ in range(NUM_USERS):
    name = random_name()
    cursor.execute('''
        INSERT INTO users (name)
        VALUES (?)
    ''', (name,))

print(f"{NUM_USERS} random users added successfully.")

# Step 2: Create the 'transaction' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        transaction_date TEXT NOT NULL,
        amount REAL NOT NULL,
        transaction_type TEXT NOT NULL CHECK(transaction_type IN ('credit', 'debit')),
        description TEXT,
        status TEXT CHECK(status IN ('pending', 'completed', 'failed')) DEFAULT 'pending',
        reference_id TEXT UNIQUE,
        FOREIGN KEY(user_id) REFERENCES users(user_id)
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

50 random users added successfully.


In [5]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to the SQLite database
conn = sqlite3.connect('transactions.db')
cursor = conn.cursor()

# Step 1: Ensure Unique Reference IDs
used_reference_ids = set()

# Function to generate a unique reference ID
def generate_unique_reference_id():
    while True:
        reference_id = f'REF-{random.randint(100000, 999999)}'
        if reference_id not in used_reference_ids:
            used_reference_ids.add(reference_id)
            return reference_id

# Step 2: Insert 1000 random transactions into the 'transactions' table
NUM_TRANSACTIONS = 1000
NUM_USERS = 50
HIGH_SPENDING_PERCENTAGE = 0.25
MEAN_AMOUNT = 50  # Assume a mean amount of $50 for normal transactions
HIGH_AMOUNT_MULTIPLIER = 2  # 2x higher for unusual transactions
TRANSACTION_TYPES = ['credit', 'debit']
STATUS_OPTIONS = ['pending', 'completed', 'failed']

# Function to generate a random date within the past year
def random_date_within_last_year():
    today = datetime.today()
    random_days_ago = random.randint(0, 365)
    return today - timedelta(days=random_days_ago)

# Insert random transactions into the 'transactions' table
for _ in range(NUM_TRANSACTIONS):
    user_id = random.randint(1, NUM_USERS)  # Use the user_id from the users table
    transaction_date = random_date_within_last_year().strftime('%Y-%m-%d')
    transaction_type = random.choice(TRANSACTION_TYPES)
    description = f'Transaction for user {user_id}'
    status = random.choice(STATUS_OPTIONS)

    # 25% of transactions should be 2x higher than the mean
    if random.random() < HIGH_SPENDING_PERCENTAGE:
        amount = round(random.uniform(MEAN_AMOUNT * 2, MEAN_AMOUNT * 4), 2)
    else:
        amount = round(random.uniform(MEAN_AMOUNT * 0.5, MEAN_AMOUNT * 1.5), 2)

    # Generate a unique reference ID
    reference_id = generate_unique_reference_id()

    # Insert transaction into the table
    cursor.execute('''
        INSERT INTO transactions (user_id, transaction_date, amount, transaction_type, description, status, reference_id)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (user_id, transaction_date, amount, transaction_type, description, status, reference_id))

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"{NUM_TRANSACTIONS} random transactions added successfully.")


1000 random transactions added successfully.


In [35]:
import os
from langchain_groq import ChatGroq

groq_api_key="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"

# st.title("Objectbox VectorstoreDB With Llama3 Demo")

llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

In [36]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///transactions.db", sample_rows_in_table_info = 3)

In [37]:
print(db.table_info)



CREATE TABLE transactions (
	transaction_id INTEGER, 
	user_id INTEGER NOT NULL, 
	transaction_date TEXT NOT NULL, 
	amount REAL NOT NULL, 
	transaction_type TEXT NOT NULL, 
	description TEXT, 
	status TEXT DEFAULT 'pending', 
	reference_id TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY(user_id) REFERENCES users (user_id), 
	UNIQUE (reference_id)
)

/*
3 rows from transactions table:
transaction_id	user_id	transaction_date	amount	transaction_type	description	status	reference_id
1	11	2023-12-02	25.18	debit	Transaction for user 11	completed	REF-572089
2	3	2023-12-22	122.95	credit	Transaction for user 3	pending	REF-876975
3	31	2024-05-08	55.65	debit	Transaction for user 31	completed	REF-336198
*/


CREATE TABLE users (
	user_id INTEGER, 
	name TEXT NOT NULL, 
	PRIMARY KEY (user_id)
)

/*
3 rows from users table:
user_id	name
1	FjnlmUb
2	nwzFlxT
3	vhdqIGz
*/


In [38]:
system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.

You have access to the following tools for interacting with the database:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
If you see you are repeating yourself, just provide final answer and exit.

Here are some examples of user inputs and their corresponding SQL queries:"""

In [39]:
# from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# dynamic_few_shot_prompt = FewShotPromptTemplate(
#     example_selector = example_selector,
#     example_prompt=PromptTemplate.from_template(
#         "User input: {input}\nSQL query: {query}"
#     ),
#     input_variables=["input"],
#     prefix=system_prefix,
#     suffix=""
# )

In [40]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prefix),
        ("human", "{input}"),
        ("system", "{agent_scratchpad}"),
    ]
)

In [41]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, InfoSQLDatabaseTool, ListSQLDatabaseTool, QuerySQLCheckerTool

tools = [QuerySQLDataBaseTool(db = db), InfoSQLDatabaseTool(db = db), ListSQLDatabaseTool(db = db), QuerySQLCheckerTool(db = db, llm = llm)]
print(QuerySQLDataBaseTool(db = db).description)
input_element = {
        "input": "How many transactions happened in last three months?",
        "tool_names" : [tool.name for tool in tools],
        "tools" : [tool.name + " - " + tool.description.strip() for tool in tools],
        "agent_scratchpad": [],
        "system_prefix": system_prefix
    }
prompt_val = full_prompt.invoke(
    input_element
)

print(prompt_val.to_string())


    Execute a SQL query against the database and get back the result..
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    
System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.

You have access to the following tools for interacting with the database:

['sql_db_query - Execute a SQL query against the database and get back the result..\n    If the query is not correct, an error message will be returned.\n    

In [42]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, full_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)

In [43]:
last_k_messages = 4


from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id):
    chat_message_history = SQLChatMessageHistory(
    session_id=session_id, connection = "sqlite:///memory.db", table_name = "local_table"
    )

    messages = chat_message_history.get_messages()
    chat_message_history.clear()
    
    for message in messages[-last_k_messages:]:
        chat_message_history.add_message(message)
    
    print("chat_message_history ", chat_message_history)
    return chat_message_history


from langchain_core.runnables.history import RunnableWithMessageHistory

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [44]:
import gradio as gr
import uuid


with gr.Blocks() as demo:
    
    state = gr.State("")
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])


    def respond(message, chatbot_history, session_id):
        if not chatbot_history:
            session_id = uuid.uuid4().hex

        print("Session ID: ", session_id)

        response = agent_with_chat_history.invoke(
                                        {"input": message},
                                        {"configurable": {"session_id": session_id}},
                                        )

        chatbot_history.append((message, response['output']))
        return "", chatbot_history, session_id

    msg.submit(respond, [msg, chatbot, state], [msg, chatbot, state])

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [46]:
agent_executor.invoke({
    'input': "Give me a count of all the transactions"
})

{'input': 'Give me a count of all the transactions',
 'output': 'Agent stopped due to iteration limit or time limit.'}